In [1]:
"""
Create and operate on grids and profiles.

**Grid generation**

* :func:`~fatiando.gridder.regular`
* :func:`~fatiando.gridder.scatter`

**Grid operations**

* :func:`~fatiando.gridder.cut`
* :func:`~fatiando.gridder.profile`

**Interpolation**

* :func:`~fatiando.gridder.interp`
* :func:`~fatiando.gridder.interp_at`
* :func:`~fatiando.gridder.extrapolate_nans`

**Input/Output**

* :func:`~fatiando.gridder.load_surfer`: Read a Surfer grid file and return
  three 1d numpy arrays and the grid shape

**Misc**

* :func:`~fatiando.gridder.spacing`

----

"""

import numpy
import scipy.interpolate
import matplotlib.mlab


def load_surfer(fname, fmt='ascii'):
    """
    Read a Surfer grid file and return three 1d numpy arrays and the grid shape

    Surfer is a contouring, gridding and surface mapping software
    from GoldenSoftware. The names and logos for Surfer and Golden
    Software are registered trademarks of Golden Software, Inc.

    http://www.goldensoftware.com/products/surfer

    According to Surfer structure, x and y are horizontal and vertical
    screen-based coordinates respectively. If the grid is in geographic
    coordinates, x will be longitude and y latitude. If the coordinates
    are cartesian, x will be the easting and y the norting coordinates.

    WARNING: This is opposite to the convention used for Fatiando.
    See io_surfer.py in cookbook.

    Parameters:

    * fname : str
        Name of the Surfer grid file
    * fmt : str
        File type, can be 'ascii' or 'binary'

    Returns:

    * x : 1d-array
        Value of the horizontal coordinate of each grid point.
    * y : 1d-array
        Value of the vertical coordinate of each grid point.
    * grd : 1d-array
        Values of the field in each grid point. Field can be for example
        topography, gravity anomaly etc
    * shape : tuple = (ny, nx)
        The number of points in the vertical and horizontal grid dimensions,
        respectively

    """
    assert fmt in ['ascii', 'binary'], "Invalid grid format '%s'. Should be \
        'ascii' or 'binary'." % (fmt)
    if fmt == 'ascii':
        # Surfer ASCII grid structure
        # DSAA            Surfer ASCII GRD ID
        # nCols nRows     number of columns and rows
        # xMin xMax       X min max
        # yMin yMax       Y min max
        # zMin zMax       Z min max
        # z11 z21 z31 ... List of Z values
        with open(fname) as ftext:
            # DSAA is a Surfer ASCII GRD ID
            id = ftext.readline()
            # Read the number of columns (nx) and rows (ny)
            nx, ny = [int(s) for s in ftext.readline().split()]
            # Read the min/max value of x (columns/longitue)
            xmin, xmax = [float(s) for s in ftext.readline().split()]
            # Read the min/max value of  y(rows/latitude)
            ymin, ymax = [float(s) for s in ftext.readline().split()]
            # Read the min/max value of grd
            zmin, zmax = [float(s) for s in ftext.readline().split()]
            data = numpy.fromiter((float(i) for line in ftext for i in
                                   line.split()), dtype='f')
            grd = numpy.ma.masked_greater_equal(data, 1.70141e+38)
        # Create x and y numpy arrays
        x = numpy.linspace(xmin, xmax, nx)
        y = numpy.linspace(ymin, ymax, ny)
        x, y = [tmp.ravel() for tmp in numpy.meshgrid(x, y)]
    if fmt == 'binary':
        raise NotImplementedError(
            "Binary file support is not implemented yet.")
    return x, y, grd, (ny, nx)

def regular(area, shape, z=None):
    """
    Create a regular grid. Order of the output grid is x varies first, then y.

    Parameters:

    * area
        ``(x1, x2, y1, y2)``: Borders of the grid
    * shape
        Shape of the regular grid, ie ``(ny, nx)``.
    * z
        Optional. z coordinate of the grid points. If given, will return an
        array with the value *z*.

    Returns:

    * ``[xcoords, ycoords]``
        Numpy arrays with the x and y coordinates of the grid points
    * ``[xcoords, ycoords, zcoords]``
        If *z* given. Numpy arrays with the x, y, and z coordinates of the grid
        points

    """
    ny, nx = shape
    x1, x2, y1, y2 = area
    dy, dx = spacing(area, shape)
    x_range = numpy.arange(x1, x2, dx)
    y_range = numpy.arange(y1, y2, dy)
    # Need to make sure that the number of points in the grid is correct
    # because of rounding errors in arange. Sometimes x2 and y2 are included,
    # sometimes not
    if len(x_range) < nx:
        x_range = numpy.append(x_range, x2)
    if len(y_range) < ny:
        y_range = numpy.append(y_range, y2)
    assert len(x_range) == nx, "Failed! x_range doesn't have nx points"
    assert len(y_range) == ny, "Failed! y_range doesn't have ny points"
    xcoords, ycoords = [mat.ravel()
                        for mat in numpy.meshgrid(x_range, y_range)]
    if z is not None:
        zcoords = z * numpy.ones_like(xcoords)
        return [xcoords, ycoords, zcoords]
    else:
        return [xcoords, ycoords]

def scatter(area, n, z=None, seed=None):
    """
    Create an irregular grid with a random scattering of points.

    Parameters:

    * area
        ``(x1, x2, y1, y2)``: Borders of the grid
    * n
        Number of points
    * z
        Optional. z coordinate of the points. If given, will return an
        array with the value *z*.
    * seed : None or int
        Seed used to generate the pseudo-random numbers. If `None`, will use a
        different seed every time. Use the same seed to generate the same
        random points.

    Returns:

    * ``[xcoords, ycoords]``
        Numpy arrays with the x and y coordinates of the points
    * ``[xcoords, ycoords, zcoords]``
        If *z* given. Arrays with the x, y, and z coordinates of the points

    """
    x1, x2, y1, y2 = area
    numpy.random.seed(seed)
    xcoords = numpy.random.uniform(x1, x2, n)
    ycoords = numpy.random.uniform(y1, y2, n)
    numpy.random.seed()
    if z is not None:
        zcoords = z * numpy.ones(n)
        return [xcoords, ycoords, zcoords]
    else:
        return [xcoords, ycoords]

def spacing(area, shape):
    """
    Returns the spacing between grid nodes

    Parameters:

    * area
        ``(x1, x2, y1, y2)``: Borders of the grid
    * shape
        Shape of the regular grid, ie ``(ny, nx)``.

    Returns:

    * ``[dy, dx]``
        Spacing the y and x directions

    """
    x1, x2, y1, y2 = area
    ny, nx = shape
    dx = float(x2 - x1) / float(nx - 1)
    dy = float(y2 - y1) / float(ny - 1)
    return [dy, dx]

def interp(x, y, v, shape, area=None, algorithm='cubic', extrapolate=False):
    """
    Interpolate data onto a regular grid.

    Parameters:

    * x, y : 1D arrays
        Arrays with the x and y coordinates of the data points.
    * v : 1D array
        Array with the scalar value assigned to the data points.
    * shape : tuple = (ny, nx)
        Shape of the interpolated regular grid, ie (ny, nx).
    * area : tuple = (x1, x2, y1, y2)
        The are where the data will be interpolated. If None, then will get the
        area from *x* and *y*.
    * algorithm : string
        Interpolation algorithm. Either ``'cubic'``, ``'nearest'``,
        ``'linear'`` (see scipy.interpolate.griddata), or ``'nn'`` for nearest
        neighbors (using matplotlib.mlab.griddata)
    * extrapolate : True or False
        If True, will extrapolate values outside of the convex hull of the data
        points.

    Returns:

    * ``[x, y, v]``
        Three 1D arrays with the interpolated x, y, and v

    """
    if algorithm not in ['cubic', 'linear', 'nearest', 'nn']:
        raise ValueError("Invalid interpolation algorithm: " + str(algorithm))
    ny, nx = shape
    if area is None:
        area = (x.min(), x.max(), y.min(), y.max())
    x1, x2, y1, y2 = area
    xs = numpy.linspace(x1, x2, nx)
    ys = numpy.linspace(y1, y2, ny)
    xp, yp = [i.ravel() for i in numpy.meshgrid(xs, ys)]
    if algorithm == 'nn':
        grid = matplotlib.mlab.griddata(x, y, v, numpy.reshape(xp, shape),
                                        numpy.reshape(yp, shape),
                                        interp='nn').ravel()
        if extrapolate and numpy.ma.is_masked(grid):
            grid = extrapolate_nans(xp, yp, grid)
    else:
        grid = interp_at(x, y, v, xp, yp, algorithm=algorithm,
                         extrapolate=extrapolate)
    return [xp, yp, grid]


def interp_at(x, y, v, xp, yp, algorithm='cubic', extrapolate=False):
    """
    Interpolate data onto the specified points.

    Parameters:

    * x, y : 1D arrays
        Arrays with the x and y coordinates of the data points.
    * v : 1D array
        Array with the scalar value assigned to the data points.
    * xp, yp : 1D arrays
        Points where the data values will be interpolated
    * algorithm : string
        Interpolation algorithm. Either ``'cubic'``, ``'nearest'``,
        ``'linear'`` (see scipy.interpolate.griddata)
    * extrapolate : True or False
        If True, will extrapolate values outside of the convex hull of the data
        points.

    Returns:

    * v : 1D array
        1D array with the interpolated v values.

    """
    if algorithm not in ['cubic', 'linear', 'nearest']:
        raise ValueError("Invalid interpolation algorithm: " + str(algorithm))
    grid = scipy.interpolate.griddata((x, y), v, (xp, yp),
                                      method=algorithm).ravel()
    if extrapolate and algorithm != 'nearest' and numpy.any(numpy.isnan(grid)):
        grid = extrapolate_nans(xp, yp, grid)
    return grid

def profile(x, y, v, point1, point2, size, extrapolate=False):
    """
    Extract a data profile between 2 points.

    Uses interpolation to calculate the data values at the profile points.

    Parameters:

    * x, y : 1D arrays
        Arrays with the x and y coordinates of the data points.
    * v : 1D array
        Array with the scalar value assigned to the data points.
    * point1, point2 : lists = [x, y]
        Lists the x, y coordinates of the 2 points between which the profile
        will be extracted.
    * size : int
        Number of points along the profile.
    * extrapolate : True or False
        If True, will extrapolate values outside of the convex hull of the data
        points.

    Returns:

    * [xp, yp, distances, vp] : 1d arrays
        ``xp`` and ``yp`` are the x, y coordinates of the points along the
        profile.
        ``distances`` are the distances of the profile points to ``point1``
        ``vp`` are the data points along the profile.

    """
    x1, y1 = point1
    x2, y2 = point2
    maxdist = numpy.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    distances = numpy.linspace(0, maxdist, size)
    angle = numpy.arctan2(y2 - y1, x2 - x1)
    xp = x1 + distances * numpy.cos(angle)
    yp = y1 + distances * numpy.sin(angle)
    vp = interp_at(x, y, v, xp, yp, algorithm='cubic', extrapolate=extrapolate)
    return xp, yp, distances, vp


def extrapolate_nans(x, y, v):
    """"
    Extrapolate the NaNs or masked values in a grid INPLACE using nearest
    value.

    .. warning:: Replaces the NaN or masked values of the original array!

    Parameters:

    * x, y : 1D arrays
        Arrays with the x and y coordinates of the data points.
    * v : 1D array
        Array with the scalar value assigned to the data points.

    Returns:

    * v : 1D array
        The array with NaNs or masked values extrapolated.

    """
    if numpy.ma.is_masked(v):
        nans = v.mask
    else:
        nans = numpy.isnan(v)
    notnans = numpy.logical_not(nans)
    v[nans] = scipy.interpolate.griddata((x[notnans], y[notnans]), v[notnans],
                                         (x[nans], y[nans]),
                                         method='nearest').ravel()
    return v


def cut(x, y, scalars, area):
    """
    Return a subsection of a grid.

    The returned subsection is not a copy! In technical terms, returns a slice
    of the numpy arrays. So changes made to the subsection reflect on the
    original grid. Use numpy.copy to make copies of the subsections and avoid
    this.

    Parameters:

    * x, y
        Arrays with the x and y coordinates of the data points.
    * scalars
        List of arrays with the scalar values assigned to the grid points.
    * area
        ``(x1, x2, y1, y2)``: Borders of the subsection

    Returns:

    * ``[subx, suby, subscalars]``
        Arrays with x and y coordinates and scalar values of the subsection.

    """
    xmin, xmax, ymin, ymax = area
    if len(x) != len(y):
        raise ValueError("x and y must have the same length")
    inside = [i for i in xrange(len(x))
              if x[i] >= xmin and x[i] <= xmax
              and y[i] >= ymin and y[i] <= ymax]
    return [x[inside], y[inside], [s[inside] for s in scalars]]

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj
from shapely import geometry
import matplotlib.pyplot as plt

gdb='/home/ggrl/geodatabase/'

%matplotlib widget

from sources import source_code as f

## Interpolando Gama Line

In [4]:
dic_cartas, dic_raw_meta = f.get_region(escala='100k',id='SC23_Z_A_IV',geof='gama_line_1089',camada='litologia_100k')

importando dados geofisicos


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


# --- Iniciando seleção de área de estudo
1 folha cartográfica selecionada

Lista de chaves do dicionario de cartas: ['region', 'region_proj', 'raw_data', 'splines', 'scores', 'lito_splines', 'mean_score', 'litologia', 'cubic', 'lito_cubic']
Listando atributos dos dados geofisicos


100%|██████████| 12/12 [00:00<00:00, 294337.12it/s]

# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']


Lista de chaves do dicionario de metadata bruto: ['Metadata', 'Lista_id', 'Lista_at_geof', 'Lista_at_geog', 'Lista_at_proj', 'Percentiles', 'Malha_cartografica']

# --- Início da iteração entre as folhas cartográficas #


0it [00:00, ?it/s]

SC23_Z_A_IV geometry       POLYGON ((500000 8839304.905873939, 554712.561...
region                              (-45.0, -44.5, -11.0, -10.5)
region_proj    (500000.0, 554712.5614995236, 8783975.08911734...
Name: SC23_Z_A_IV, dtype: object
# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']
 com 77643 pontos de amostragem


In [40]:
dic_cartas['litologia']

{'SC23_Z_A_IV':        ID_UNIDADE    SIGLA    HIERARQUIA  \
 2816         1006     N1dl  Não definida   
 2820         1006     N1dl  Não definida   
 2821         4873   NP1rp3       Unidade   
 2823         4876  NP1rp2q    Litofácies   
 2825         5773   PP2cmv       Unidade   
 ...           ...      ...           ...   
 54755        4873   NP1rp3       Unidade   
 54756        1006     N1dl  Não definida   
 54761        1006     N1dl  Não definida   
 54771        4873   NP1rp3       Unidade   
 54773        1006     N1dl  Não definida   
 
                                                     NOME  \
 2816         Coberturas detrito-lateríticas ferruginosas   
 2820         Coberturas detrito-lateríticas ferruginosas   
 2821                            Unidade Rio Preto, xisto   
 2823   Litofácies Rio Preto, metassedimentos quartzos...   
 2825           Unidade Cristalândia do Piauí, metamáfica   
 ...                                                  ...   
 54755          

In [42]:
geof = dic_cartas['raw_data']['SC23_Z_A_IV']
area = dic_cartas['region_proj']['SC23_Z_A_IV']

litologia = dic_cartas['litologia']['SC23_Z_A_IV']

geof['geometry'] = [geometry.Point(x, y) for x, y in zip(geof['X'], geof['Y'])]

crs = "+proj=utm +zone=23 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
gdf_geof = gpd.GeoDataFrame(geof, geometry='geometry', crs=crs)

In [43]:
area

(500000.0, 554712.5614995236, 8783975.089117344, 8839304.90587394)

In [137]:
area[0]-1000

499000.0

In [138]:
area = area[0]-1000 , area[1]+1000 , area[2]-1000, area[3]+1000

In [140]:
gdf_geof

,X,Y,MDT,KPERC,eU,eth,CTCOR,THKRAZAO,UKRAZAO,UTHRAZAO,LONGITUDE,LATITUDE,geometry
82496,500166.33,8839287.84,437.17,0.09,0.57,5.46,412.00,57.4927,6.0210,0.1047,-44.998480,-10.500154,POINT (500166.330 8839287.840)
82497,500171.98,8839210.30,436.96,0.08,0.94,5.64,466.59,71.7172,12.0187,0.1676,-44.998428,-10.500856,POINT (500171.980 8839210.300)
82498,500177.11,8839132.51,435.99,0.09,0.98,5.64,474.32,61.7654,10.7205,0.1736,-44.998381,-10.501559,POINT (500177.110 8839132.510)
82499,500181.97,8839054.61,434.94,0.15,0.72,6.19,489.18,41.4213,4.7698,0.1152,-44.998337,-10.502264,POINT (500181.970 8839054.610)
82500,500186.79,8838976.47,435.27,0.21,0.38,6.38,474.46,30.2010,1.8589,0.0616,-44.998293,-10.502970,POINT (500186.790 8838976.470)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
231927,554619.00,8838949.65,471.69,0.95,1.47,15.02,1327.04,15.8524,1.5393,0.0971,-44.500850,-10.502821,POINT (554619.000 8838949.650)
231928,554618.94,8839028.43,463.98,0.80,1.17,12.29,1087.91,15.2830,1.4522,0.0950,-44.500852,-10.502108,POINT (554618.940 8839028.430)
231929,554618.52,8839107.80,454.86,0.63,1.10,9.72,881.47,15.7108,1.7910,0.1140,-44.500857,-10.501391,POINT (554618.520 8839107.800)
231930,554617.97,8839187.24,450.46,0.57,1.19,8.10,784.10,14.0517,2.0510,0.1460,-44.500863,-10.500672,POINT (554617.970 8839187.240)


In [56]:
mag_1089 = pd.read_csv(gdb+'geof/mag_1089')

In [58]:
mag_1089_cut = mag_1089[f.vd.inside((mag_1089.X, mag_1089.Y), region = area)]
gama_10



In [60]:
#mag_1089_cut
mag_1089_cut['geometry'] = [geometry.Point(x, y) for x, y in zip(mag_1089_cut['X'], mag_1089_cut['Y'])]

crs = "+proj=utm +zone=23 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
mag_1089_gdf = gpd.GeoDataFrame(mag_1089_cut, geometry='geometry', crs=crs)

/tmp/ipykernel_2830/2125798396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mag_1089_cut['geometry'] = [geometry.Point(x, y) for x, y in zip(mag_1089_cut['X'], mag_1089_cut['Y'])]


In [111]:
# creating a grid with cells
xu, yu = regular(shape = (440, 440),
                     area = area)

#sx, sy = spacing(shape = (500, 500),area = area)

In [112]:
#lista_atributos_geof = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']

CTCOR = np.array(gdf_geof.CTCOR)

MDT = np.array(gdf_geof.MDT)

eth = np.array(gdf_geof.eth)

eU = np.array(gdf_geof.eU)

KPERC = np.array(gdf_geof.KPERC)

THKRAZAO = np.array(gdf_geof.THKRAZAO)

UKRAZAO = np.array(gdf_geof.UKRAZAO)

UTHRAZAO = np.array(gdf_geof.UTHRAZAO)

MAGIGRF = np.array(mag_1089_cut.MAGIGRF)


x2, y2 = np.array(gdf_geof.X), np.array(gdf_geof.Y)
x3, y3 = np.array(mag_1089_gdf.X), np.array(mag_1089_gdf.Y)

eth_ = interp_at(x2, y2, eth, xu, yu, algorithm = 'cubic', extrapolate = True)
eu_ = interp_at(x2, y2, eU, xu, yu, algorithm = 'cubic', extrapolate = True)
kperc_ = interp_at(x2, y2, KPERC, xu, yu, algorithm = 'cubic', extrapolate = True)
ctcor_ = interp_at(x2, y2, CTCOR, xu, yu, algorithm = 'cubic', extrapolate = True)
mdt_ = interp_at(x2, y2, MDT, xu, yu, algorithm = 'cubic', extrapolate = True)
uthrazao_ = interp_at(x2, y2, UTHRAZAO, xu, yu, algorithm = 'cubic', extrapolate = True)
ukrazao_ = interp_at(x2, y2, UKRAZAO, xu, yu, algorithm = 'cubic', extrapolate = True)
thkrazao_ = interp_at(x2, y2, THKRAZAO, xu, yu, algorithm = 'cubic', extrapolate = True)
magigrf_ = interp_at(x3, y3, MAGIGRF, xu, yu, algorithm = 'cubic', extrapolate = True)




# intialise data of lists. 
data = {'X':xu, 'Y':yu, 'MDT': mdt_,  'CTCOR': ctcor_,
        'KPERC': kperc_, 'eU':eu_, 'eth': eth_,
        'UTHRAZAO':uthrazao_,'UKRAZAO':ukrazao_,'THKRAZAO':thkrazao_,'MAGIGRF':magigrf_} 
  
# Create DataFrame 
geof_1089 = pd.DataFrame(data)
  
# Print the output. 
geof_1089

,X,Y,MDT,CTCOR,KPERC,eU,eth,UTHRAZAO,UKRAZAO,THKRAZAO,MAGIGRF
0,500000.000000,8.783975e+06,513.014076,1698.811444,1.514579,2.231990,16.860637,0.134897,1.478411,10.988898,45.813778
1,500124.629981,8.783975e+06,513.014076,1698.811444,1.514579,2.231990,16.860637,0.134897,1.478411,10.988898,45.813778
2,500249.259961,8.783975e+06,513.014076,1698.811444,1.514579,2.231990,16.860637,0.134897,1.478411,10.988898,40.564232
3,500373.889942,8.783975e+06,513.740384,1640.425429,1.470424,2.316056,15.878494,0.148822,1.586554,10.635102,35.147613
4,500498.519923,8.783975e+06,513.977322,1590.186719,1.426733,2.350087,15.148366,0.156940,1.655714,10.487335,30.456643
...,...,...,...,...,...,...,...,...,...,...,...
193595,554214.041577,8.839305e+06,481.997037,910.124992,0.666923,0.795781,10.715787,0.077176,1.208936,16.105931,33.762521
193596,554338.671558,8.839305e+06,473.282718,877.064346,0.642043,0.883981,10.062067,0.093572,1.406872,15.639826,29.602305
193597,554463.301538,8.839305e+06,463.558770,840.127921,0.611048,1.003676,9.299662,0.113786,1.667852,15.105621,26.617953
193598,554587.931519,8.839305e+06,453.295795,797.244051,0.580952,1.129925,8.415680,0.135732,1.943113,14.517530,24.389132


## Lista de Titulos para os Plots

In [147]:
list_canal = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO','MAGIGRF']
print(f"Lista de atributos: {list_canal}")

# LISTA DE TITULOS 
titles =   [
            'MDT (m)']


# DICIONARIO DE TITULOS
dic_titles = {}
for f, t in zip(list_canal, titles):
    dic_titles[f] = t

dic_titles

Lista de atributos: ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO', 'MAGIGRF']


{'MDT': 'MDT (m)'}

In [158]:
def plot_raw_data(raw_data):
    fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 12),sharex='all',sharey='all')
    
    raw_data_describe = raw_data.describe(percentiles=(0.01,0.1,0.25,0.5,0.75,0.995))

    X, Y = raw_data.X, raw_data.Y

    for ax, f in zip(axs.flat, dic_titles):
        if f == 'MDT':
            vmin = raw_data_describe[f]['1%']
            vmax = raw_data_describe[f]['99.5%']

            g = ax.scatter(c=raw_data[f], x = X,
                                        y = Y,
                                        cmap = 'terrain',
                                        s    = 2.5,
                                        vmax =vmax, vmin=vmin)

            fig.colorbar(g, ax = ax)
            ax.set_title(str(dic_titles[f]), size = 11)
            ax.axis('scaled')
        else:
            vmin = raw_data_describe[f]['1%']
            vmax = raw_data_describe[f]['99.5%']

            g = ax.scatter(c=raw_data[f], x = X,
                                        y = Y,
                                        cmap = 'hsv',
                                        s    = 2.5,
                                        vmax =vmax, vmin=vmin)

            fig.colorbar(g, ax = ax)
            ax.set_title(str(dic_titles[f]), size = 11)
            ax.axis('scaled')
    fig.delaxes(axs[0][1])
    #plt.tight_layout()

In [154]:
gdf_geof

,X,Y,MDT,KPERC,eU,eth,CTCOR,THKRAZAO,UKRAZAO,UTHRAZAO,LONGITUDE,LATITUDE,geometry
82496,500166.33,8839287.84,437.17,0.09,0.57,5.46,412.00,57.4927,6.0210,0.1047,-44.998480,-10.500154,POINT (500166.330 8839287.840)
82497,500171.98,8839210.30,436.96,0.08,0.94,5.64,466.59,71.7172,12.0187,0.1676,-44.998428,-10.500856,POINT (500171.980 8839210.300)
82498,500177.11,8839132.51,435.99,0.09,0.98,5.64,474.32,61.7654,10.7205,0.1736,-44.998381,-10.501559,POINT (500177.110 8839132.510)
82499,500181.97,8839054.61,434.94,0.15,0.72,6.19,489.18,41.4213,4.7698,0.1152,-44.998337,-10.502264,POINT (500181.970 8839054.610)
82500,500186.79,8838976.47,435.27,0.21,0.38,6.38,474.46,30.2010,1.8589,0.0616,-44.998293,-10.502970,POINT (500186.790 8838976.470)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
231927,554619.00,8838949.65,471.69,0.95,1.47,15.02,1327.04,15.8524,1.5393,0.0971,-44.500850,-10.502821,POINT (554619.000 8838949.650)
231928,554618.94,8839028.43,463.98,0.80,1.17,12.29,1087.91,15.2830,1.4522,0.0950,-44.500852,-10.502108,POINT (554618.940 8839028.430)
231929,554618.52,8839107.80,454.86,0.63,1.10,9.72,881.47,15.7108,1.7910,0.1140,-44.500857,-10.501391,POINT (554618.520 8839107.800)
231930,554617.97,8839187.24,450.46,0.57,1.19,8.10,784.10,14.0517,2.0510,0.1460,-44.500863,-10.500672,POINT (554617.970 8839187.240)


In [159]:
plot_raw_data(gdf_geof)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: 'AxesSubplot' object is not subscriptable

In [116]:
geof_1089['geometry'] = [geometry.Point(x,y) for x, y in zip(geof_1089['X'], geof_1089['Y'])]

In [117]:
gdf_1089 = gpd.GeoDataFrame(geof_1089)

In [118]:
gdf_1089.set_crs('EPSG:32723',inplace=True)

,X,Y,MDT,CTCOR,KPERC,eU,eth,UTHRAZAO,UKRAZAO,THKRAZAO,MAGIGRF,geometry
0,500000.000000,8.783975e+06,513.014076,1698.811444,1.514579,2.231990,16.860637,0.134897,1.478411,10.988898,45.813778,POINT (500000.000 8783975.089)
1,500124.629981,8.783975e+06,513.014076,1698.811444,1.514579,2.231990,16.860637,0.134897,1.478411,10.988898,45.813778,POINT (500124.630 8783975.089)
2,500249.259961,8.783975e+06,513.014076,1698.811444,1.514579,2.231990,16.860637,0.134897,1.478411,10.988898,40.564232,POINT (500249.260 8783975.089)
3,500373.889942,8.783975e+06,513.740384,1640.425429,1.470424,2.316056,15.878494,0.148822,1.586554,10.635102,35.147613,POINT (500373.890 8783975.089)
4,500498.519923,8.783975e+06,513.977322,1590.186719,1.426733,2.350087,15.148366,0.156940,1.655714,10.487335,30.456643,POINT (500498.520 8783975.089)
...,...,...,...,...,...,...,...,...,...,...,...,...
193595,554214.041577,8.839305e+06,481.997037,910.124992,0.666923,0.795781,10.715787,0.077176,1.208936,16.105931,33.762521,POINT (554214.042 8839304.906)
193596,554338.671558,8.839305e+06,473.282718,877.064346,0.642043,0.883981,10.062067,0.093572,1.406872,15.639826,29.602305,POINT (554338.672 8839304.906)
193597,554463.301538,8.839305e+06,463.558770,840.127921,0.611048,1.003676,9.299662,0.113786,1.667852,15.105621,26.617953,POINT (554463.302 8839304.906)
193598,554587.931519,8.839305e+06,453.295795,797.244051,0.580952,1.129925,8.415680,0.135732,1.943113,14.517530,24.389132,POINT (554587.932 8839304.906)


In [119]:
gdf_1089.set_geometry('geometry',inplace=True)

In [120]:
gdf_1089.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [126]:
litologia = f.importar('litologia_100k','Rio Paraim')

litologia.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [ ]:
region = dic_raw_meta['Malha_cartografica'].region[0]

litologia=litologia.cx[region[0]:region[1],region[2]:region[3]]

In [128]:
#litologia.set_crs(32723,allow_override=True,inplace=True)
litologia.to_crs(32723,inplace=True)

In [129]:
litologia.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [105]:
litologia

,ID_UNIDADE,SIGLA,HIERARQUIA,NOME,AMBIENTE_T,SUB_AMBIEN,SIGLA_PAI,NOME_PAI,LEGENDA,ESCALA,...,ERA_MIN,ERA_MAX,SISTEMA_MI,SISTEMA_MA,EPOCA_MIN,EPOCA_MAX,SIGLAS_HIS,SHAPE_AREA,SHAPE_LEN,geometry
2816,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,1:100.000,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.000041,0.037400,"POLYGON ((500176.894 8821390.751, 500170.654 8..."
2820,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,1:100.000,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.001087,0.257896,"POLYGON ((505306.762 8803062.104, 505258.182 8..."
2821,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,1:100.000,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.000170,0.094417,"POLYGON ((505145.795 8801235.292, 506323.529 8..."
2823,4876,NP1rp2q,Litofácies,"Litofácies Rio Preto, metassedimentos quartzos...",None,None,MP3NP1rp2,"MP3NP1rp2 - Unidade Rio Preto, metassedimentos...",Quartzitos a muscovita-quartzitos puros a impu...,1:100.000,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"M3N1rp2q, MP3NP1rp2q",0.000015,0.020423,"POLYGON ((509741.424 8784951.753, 509758.753 8..."
2825,5773,PP2cmv,Unidade,"Unidade Cristalândia do Piauí, metamáfica",None,None,A4PPc,A4PPc - Complexo Cristalândia do Piauí,"Corpos gabróicos a metagabros, compostos por a...",1:100.000,...,Paleoproterozóico,Paleoproterozóico,Riaciano,Riaciano,Série 3,Série 3,None,0.000323,0.077072,"POLYGON ((510857.857 8811448.300, 510816.261 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54755,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,1:100.000,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.023544,3.651426,"POLYGON ((554702.380 8832876.999, 554702.379 8..."
54756,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,1:100.000,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.001673,0.238141,"POLYGON ((554710.777 8838140.124, 554709.914 8..."
54761,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,1:100.000,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.000131,0.063211,"POLYGON ((555558.288 8838523.747, 555557.665 8..."
54771,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,1:100.000,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.008726,1.113618,"POLYGON ((554693.086 8827078.565, 554693.163 8..."


In [130]:
litologia.reset_index(inplace=True)

In [83]:
ax = litologia.plot('SIGLA')
plt.scatter(gdf_1089.X, gdf_1089.Y, cmap='hsv', s=5, c=gdf_1089.eth)
plt.colorbar(label=u'eTh')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(12, 8)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
geof_1089['closest_unid'] = gdf_1089['geometry'].apply(lambda x: litologia['SIGLA'].iloc[litologia.distance(x).idxmin()])

In [134]:
geof_1089.to_csv(gdb+'dataframe_input_model/geof_1089_SC23_Z_A_IV_125m.csv')